# Sample Maps with shared utils

In [ ]:
# In terminal - try to get ipyleaflet to render (but this still doesn't work)
#jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet js

In [1]:
import branca
import geopandas as gpd
import pandas as pd

import shared_utils 

In [2]:
shared_utils.geography_utils.CA_NAD83Albers

'EPSG:3310'

In [3]:
GEOSPATIAL_DATA = "https://opendata.arcgis.com/datasets/a0f1da19bffe471ea89bd6a241e5604b_0.geojson"
gdf = gpd.read_file(GEOSPATIAL_DATA)
gdf

,OBJECTID,DISTRICT,Region,Shape_Length,Shape_Area,geometry
0,1,1,North,1.913829e+06,4.188500e+10,"POLYGON ((-123.51819 42.00031, -123.51794 42.0..."
1,2,2,North,1.776128e+06,1.266855e+11,"POLYGON ((-122.30683 42.00816, -122.32039 42.0..."
2,3,3,North,1.467236e+06,5.459471e+10,"POLYGON ((-121.40463 40.14665, -121.40452 40.1..."
3,4,4,,1.540172e+06,3.123284e+10,"POLYGON ((-122.35190 38.83627, -122.35432 38.8..."
4,5,5,Central,1.593306e+06,4.341220e+10,"POLYGON ((-122.15057 37.27596, -122.15058 37.2..."
5,6,6,Central,1.457489e+06,7.768882e+10,"POLYGON ((-119.27249 37.73954, -119.27299 37.7..."
6,7,8,,1.365819e+06,1.049158e+11,"POLYGON ((-115.66384 35.80924, -115.68169 35.8..."
7,8,10,Central,1.199090e+06,4.591954e+10,"POLYGON ((-119.88972 38.92320, -119.89712 38.9..."
8,9,11,,1.104659e+06,3.221576e+10,"POLYGON ((-117.36428 33.50503, -117.43208 33.5..."
9,10,12,,3.611343e+05,2.983890e+09,"MULTIPOLYGON (((-117.70373 33.46011, -117.7039..."


In [ ]:
print(f"original CRS: {gdf.crs}")
gdf = gdf.to_crs(shared_utils.geography_utils.CA_StatePlane)
print(f"new CRS: {gdf.crs}")

In [ ]:
shared_utils.utils.geoparquet_gcs_export?

In [ ]:
GCS_FILE_PATH = "gs://calitp-analytics-data/"
FILE_NAME = "test_geoparquet"
shared_utils.utils.geoparquet_gcs_export(gdf, GCS_FILE_PATH, FILE_NAME)

In [ ]:
popup_dict = {
    "DISTRICT": "Caltrans District",
    "Region": "Caltrans Region",
}

'''
# Set your own step colormap and specify colors and cut-offs
colorscale = branca.colormap.StepColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

# Set your own linear colormap and specify colors and cut-offs
colorscale = branca.colormap.LinearColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

# Use preset step colormap
colorscale = branca.colormap.step.Accent_03

# Use preset linear colormap
colorscale = branca.colormap.linear.Blues_06

'''
colorscale = (branca.colormap.step.Accent_07
              .scale(vmin=gdf.DISTRICT.min(), 
                     vmax=gdf.DISTRICT.max())
             )

m = shared_utils.map_utils.make_folium_choropleth_map(
    gdf,
    plot_col = "DISTRICT",
    popup_dict = popup_dict,
    tooltip_dict = popup_dict,
    colorscale = colorscale,
    fig_width = 500, fig_height = 800,
    zoom = shared_utils.map_utils.REGION_CENTROIDS["CA"][1],
    centroid = shared_utils.map_utils.REGION_CENTROIDS["CA"][0],
    title="CA Map"
)

#m

In [4]:
gdf = gdf.to_crs(shared_utils.geography_utils.WGS84)

choropleth_dict = {
    "layer_name": "Area",
    "MIN_VALUE": gdf.Shape_Area.min(),
    "MAX_VALUE": gdf.Shape_Area.max(),
    "plot_col_name": "Area (sq mi)",
    "fig_width": '100%',
    "fig_height": '100%',
    "fig_min_width_px": '400px',
    "fig_min_height_px": '600px',
}

colorscale = branca.colormap.linear.Greens_09

f = shared_utils.map_utils.make_ipyleaflet_choropleth_map(
    gdf,
    geometry_col = "DISTRICT", 
    plot_col = "Shape_Area",
    choropleth_dict = choropleth_dict,
    colorscale = colorscale
)

f

Map(center=[35.8, -119.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…